In [1]:
# 全局攻击 逃逸攻击 数据集：异配图数据集；代理模型：GCN；攻击算法：DICE
import torch
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
import torch_geometric.transforms as T
import torch.optim as optim
import scipy.sparse as sp
from copy import deepcopy
from torch_geometric.utils import to_undirected
from torch_sparse import SparseTensor
from tqdm import tqdm

from utils import *
from H2GCN import H2GCN
from gcn import GCN
from DICE import DICE

Using backend: pytorch


In [2]:
# 模拟命令行参数
class Args:
    def __init__(self, dataset='cora'):
        self.seed = 15
        self.dataset = dataset
        self.input_size = 0
        self.output_size = 0
        self.hidden_size = 64
        self.epochs = 2000
        self.lr = 0.001
        self.drop_prob = 0.5
        self.round = 2
        self.train_ratio = 0.6
        self.patience = 200

# 传入特定的参数值
args = Args(dataset='citeseer') 
# 默认dataset为'cora'，可以传入的dataset参数有：choices=['cora', 'citeseer', 'pubmed', 'film', 'squirrel', 'chameleon', 'texas', 'cornell', 'wisconsin']

In [3]:
# 是否使用cuda
args.cuda = torch.cuda.is_available()
print('cuda: %s' % args.cuda)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

np.random.seed(args.seed)
torch.manual_seed(args.seed)
if args.cuda:
    torch.cuda.manual_seed(args.seed)

cuda: True


In [4]:
# 加载数据集
g, nclass, features, labels, train, val, test = preprocess_data(args.dataset, args.train_ratio)

features = features.to(device)
labels = labels.to(device)

idx_train = train.numpy()
idx_test = test.numpy()
idx_val = val.numpy()

train = train.to(device)
test = test.to(device)
val = val.to(device)

citeseer 6


/home/wangyajing/experiment/utils.py:220: RuntimeWarning: divide by zero encountered in power
  r_inv = np.power(rowsum, -1).flatten()


In [5]:
node_classes=labels
# 使用torch.unique函数获取唯一值
unique_classes = torch.unique(node_classes)
# 获取唯一值的数量
c = len(unique_classes)
num_loops = 2
perturbed_rate = 0.1 # 扰动率
print("一共有 {} 个类别。".format(c))

一共有 6 个类别。


In [6]:
def trans1(features):
    # 转features数据格式
    # 获取非零元素的索引和对应的值
    indices = torch.nonzero(features).t()
    values = features[indices[0], indices[1]]
    # 构建CSR格式的三个数组
    row_ptr = torch.tensor([0] + torch.cumsum(torch.bincount(indices[0]), dim=0).tolist(), dtype=torch.int64)
    col_indices = indices[1]
    data = values
    # 创建CSR格式的稀疏矩阵
    features = sp.csr_matrix((data.cpu().numpy(), col_indices.cpu().numpy(), row_ptr.cpu().numpy()), shape=features.shape)
   
    return features

def trans2(features):
    # 将 CSR 格式的稀疏矩阵转换成 PyTorch 稀疏张量
    coo_matrix = features.tocoo()
    indices = torch.tensor([coo_matrix.row, coo_matrix.col], dtype=torch.long)
    values = torch.tensor(coo_matrix.data, dtype=torch.float32)
    size = coo_matrix.shape

    # 创建 PyTorch 稀疏张量
    sparse_tensor = torch.sparse_coo_tensor(indices, values, size=size)

    return sparse_tensor

In [7]:
def test(adj):
    ''' test on GCN '''
    global gcn
    # adj = normalize_adj_tensor(adj)
    gcn = GCN(nfeat=features.shape[1],
              nhid=16,
              nclass=labels.max().item() + 1,
              dropout=0.5, device=device)

    gcn = gcn.to(device)

    gcn.fit(features, adj, labels, idx_train) # train without model picking
    # gcn.fit(features, adj, labels, idx_train, idx_val) # train with validation model picking
    output = gcn.output
    loss_test = F.nll_loss(output[idx_test], labels[idx_test])
    acc_test = accuracy(output[idx_test], labels[idx_test])
    print("Test set results:",
          "loss= {:.4f}".format(loss_test.item()),
          "accuracy= {:.4f}".format(acc_test.item()))

    return acc_test.item()

In [8]:
def test(adj_ori):
    ''' test on GCN '''
    global gcn
    # adj = normalize_adj_tensor(adj)
    gcn = GCN(nfeat=features.shape[1],
              nhid=16,
              nclass=labels.max().item() + 1,
              dropout=0.5, device=device)

    gcn = gcn.to(device)

    gcn.fit(features, adj_ori, labels, idx_train) # train without model picking
    # gcn.fit(features, adj, labels, idx_train, idx_val) # train with validation model picking
    output = gcn.output
    loss_test = F.nll_loss(output[idx_test], labels[idx_test])
    acc_test = accuracy(output[idx_test], labels[idx_test])
    info = 0
    for i in range(c):
        id_lables = idx_test[np.where(labels[idx_test].to('cpu') == i)] # 标签为0的节点id
        p = accuracy(output[id_lables], labels[id_lables])
        p = p.to('cpu')
        print(p)
        info += (-p) * np.log(p)
    
    print("Test set results:",
          "loss= {:.4f}".format(loss_test.item()),
          "accuracy= {:.4f}".format(acc_test.item()))
    print(info)

    return acc_test.item()

In [9]:
# 转换为 CSR matrix，原图
adj_clean = g.adjacency_matrix(transpose=True, scipy_fmt="csr")
adj_clean = adj_clean.astype(np.float32)

features_ori =features
features = trans1(features)
features = trans2(features)

In [10]:
adj = trans2(adj_clean)
print('=== testing GCN on clean graph ===')
test(adj)

=== testing GCN on clean graph ===
tensor(0.4156, dtype=torch.float64)
tensor(0.6374, dtype=torch.float64)
tensor(0.7072, dtype=torch.float64)
tensor(0.7316, dtype=torch.float64)
tensor(0.8343, dtype=torch.float64)
tensor(0.8250, dtype=torch.float64)
Test set results: loss= 1.0617 accuracy= 0.7180
tensor(1.4355, dtype=torch.float64)


0.718

In [11]:
# 设置攻击模型
target_node = np.arange(adj_clean.shape[0])         # 可以修改目标节点ID
#n_edge_mod = int(g.num_edges() * perturbed_rate)
n_edge_mod = 10
ratio_delete = 0.6
model = DICE(n_edge_mod, ratio_delete)

In [12]:
# 进行攻击
modified_adj = model.attack(adj_clean, idx_test, labels)  # 直接攻击

modified_adj = trans2(modified_adj)
modified_adj = modified_adj.to(device)

Delete internally......


  0%|          | 0/2790 [00:00<?, ?it/s]

Connect externally......


  0%|          | 0/1000 [00:00<?, ?it/s]

DICE attack finished. 6 edges were removed, 4 edges were connected.


In [13]:
# 逃逸攻击
print('=== testing GCN on perturbed graph ===')
gcn.eval()
with torch.no_grad():
    logits = gcn(features_ori, modified_adj)
pred = logits.argmax(dim=-1)
test_acc = (pred[idx_test] == labels[idx_test]).float().mean()
print(test_acc)

=== testing GCN on perturbed graph ===
tensor(0.6550, device='cuda:0')
